# IPL Win Probability Predictor

### Import libraires

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
matches=pd.read_csv("matches.csv")
deliveries=pd.read_csv("deliveries.csv")

In [2]:
matches.shape

(1095, 20)

In [3]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1095 non-null   int64  
 1   season           1095 non-null   object 
 2   city             1044 non-null   object 
 3   date             1095 non-null   object 
 4   match_type       1095 non-null   object 
 5   player_of_match  1090 non-null   object 
 6   venue            1095 non-null   object 
 7   team1            1095 non-null   object 
 8   team2            1095 non-null   object 
 9   toss_winner      1095 non-null   object 
 10  toss_decision    1095 non-null   object 
 11  winner           1090 non-null   object 
 12  result           1095 non-null   object 
 13  result_margin    1076 non-null   float64
 14  target_runs      1092 non-null   float64
 15  target_overs     1092 non-null   float64
 16  super_over       1095 non-null   object 
 17  method        

In [4]:
deliveries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260920 entries, 0 to 260919
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   match_id          260920 non-null  int64 
 1   inning            260920 non-null  int64 
 2   batting_team      260920 non-null  object
 3   bowling_team      260920 non-null  object
 4   over              260920 non-null  int64 
 5   ball              260920 non-null  int64 
 6   batter            260920 non-null  object
 7   bowler            260920 non-null  object
 8   non_striker       260920 non-null  object
 9   batsman_runs      260920 non-null  int64 
 10  extra_runs        260920 non-null  int64 
 11  total_runs        260920 non-null  int64 
 12  extras_type       14125 non-null   object
 13  is_wicket         260920 non-null  int64 
 14  player_dismissed  12950 non-null   object
 15  dismissal_kind    12950 non-null   object
 16  fielder           9354 non-null    obj

keep column
- batting_team
- bowling_team
- city
- run_left
- balls_left
- wickets_left
- total_runs

In [5]:
total_score_df=deliveries.groupby(["match_id", "inning"]).sum()['total_runs'].reset_index()

In [6]:
total_score_df

,match_id,inning,total_runs
0,335982,1,222
1,335982,2,82
2,335983,1,240
3,335983,2,207
4,335984,1,129
...,...,...,...
2212,1426310,2,174
2213,1426311,1,175
2214,1426311,2,139
2215,1426312,1,113


In [7]:
total_score_df=total_score_df[total_score_df['inning']==1]

In [8]:
total_score_df

,match_id,inning,total_runs
0,335982,1,222
2,335983,1,240
4,335984,1,129
6,335985,1,165
8,335986,1,110
...,...,...,...
2207,1426307,1,214
2209,1426309,1,159
2211,1426310,1,172
2213,1426311,1,175


In [9]:
match_df=matches.merge(total_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')

In [10]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1095 non-null   int64  
 1   season           1095 non-null   object 
 2   city             1044 non-null   object 
 3   date             1095 non-null   object 
 4   match_type       1095 non-null   object 
 5   player_of_match  1090 non-null   object 
 6   venue            1095 non-null   object 
 7   team1            1095 non-null   object 
 8   team2            1095 non-null   object 
 9   toss_winner      1095 non-null   object 
 10  toss_decision    1095 non-null   object 
 11  winner           1090 non-null   object 
 12  result           1095 non-null   object 
 13  result_margin    1076 non-null   float64
 14  target_runs      1092 non-null   float64
 15  target_overs     1092 non-null   float64
 16  super_over       1095 non-null   object 
 17  method        

In [11]:
matches['team1'].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab',
       'Delhi Daredevils', 'Mumbai Indians', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Deccan Chargers', 'Chennai Super Kings',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Gujarat Lions', 'Rising Pune Supergiants',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans',
       'Royal Challengers Bengaluru'], dtype=object)

In [12]:
teams=['Royal Challengers Bangalore',
'Mumbai Indians',
'Kolkata Knight Riders',
'Rajasthan Royals',
'Chennai Super Kings',
'Sunrisers Hyderabad',
'Delhi Capitals',
'Punjab Kings',
'Lucknow Super Giants',
'Gujarat Titans']

In [13]:
match_df['team1']=match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2']=match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['team1']=match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2']=match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [14]:
match_df=match_df[match_df['team1'].isin(teams)]
match_df=match_df[match_df['team2'].isin(teams)]

In [15]:
match_df.head(10)

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,...,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2,match_id,total_runs
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,...,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen,335982,222
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,...,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar,335984,129
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,...,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper,335985,165
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Sunrisers Hyderabad,Deccan Chargers,...,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan,335986,110
6,335988,2007/08,Hyderabad,2008-04-22,League,V Sehwag,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Delhi Capitals,Deccan Chargers,...,wickets,9.0,143.0,20.0,N,NaN,IL Howell,AM Saheba,335988,142
7,335989,2007/08,Chennai,2008-04-23,League,ML Hayden,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Mumbai Indians,Mumbai Indians,...,runs,6.0,209.0,20.0,N,NaN,DJ Harper,GA Pratapkumar,335989,208
8,335990,2007/08,Hyderabad,2008-04-24,League,YK Pathan,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Rajasthan Royals,Rajasthan Royals,...,wickets,3.0,215.0,20.0,N,NaN,Asad Rauf,MR Benson,335990,214
10,335992,2007/08,Bangalore,2008-04-26,League,SR Watson,M Chinnaswamy Stadium,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,...,wickets,7.0,136.0,20.0,N,NaN,MR Benson,IL Howell,335992,135
11,335993,2007/08,Chennai,2008-04-26,League,JDP Oram,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Kolkata Knight Riders,Kolkata Knight Riders,...,wickets,9.0,148.0,20.0,N,NaN,BF Bowden,AV Jayaprakash,335993,147
12,335994,2007/08,Mumbai,2008-04-27,League,AC Gilchrist,Dr DY Patil Sports Academy,Mumbai Indians,Sunrisers Hyderabad,Deccan Chargers,...,wickets,10.0,155.0,20.0,N,NaN,Asad Rauf,SL Shastri,335994,154


In [16]:
match_df['method'].value_counts()

method
D/L    11
Name: count, dtype: int64

In [17]:
match_df = match_df[match_df['method'] != 'D/L']

In [18]:
match_df.shape

(779, 22)

In [19]:
match_df=match_df[['match_id','city','winner','total_runs']]

In [20]:
deliveries_df=match_df.merge(deliveries,on='match_id')

In [21]:
deliveries_df=deliveries_df[deliveries_df['inning']==2]

In [22]:
deliveries_df.shape

(90267, 20)

In [23]:
deliveries_df['total_runs_y'] = pd.to_numeric(deliveries_df['total_runs_y'], errors='coerce')

In [24]:
deliveries_df['current_score'] = deliveries_df.groupby('match_id')['total_runs_y'].cumsum()

In [25]:
deliveries_df['current_score']

124         1
125         2
126         2
127         3
128         4
         ... 
186720    110
186721    111
186722    112
186723    113
186724    114
Name: current_score, Length: 90267, dtype: int64

In [26]:
deliveries_df['runs_left']=deliveries_df['total_runs_x']-deliveries_df['current_score']

In [27]:
deliveries_df['balls_left']=126-deliveries_df['over']*6+deliveries_df['ball']

In [28]:
deliveries_df['player_dismissed']=deliveries_df['player_dismissed'].fillna("0")
# Convert player_dismissed to numeric (0 for no wicket, 1 for wicket)
deliveries_df['player_dismissed'] = deliveries_df['player_dismissed'].apply(lambda x: 0 if pd.isna(x) else 1)

# Ensure column is integer type
deliveries_df['player_dismissed'] = deliveries_df['player_dismissed'].astype(int)

# Now calculate cumulative wickets per match
deliveries_df['wickets_fallen'] = deliveries_df.groupby('match_id')['player_dismissed'].cumsum()


In [29]:
# Convert player_dismissed to numeric (0 for no wicket, 1 for wicket)
deliveries_df['player_dismissed'] = deliveries_df['player_dismissed'].apply(lambda x: 0 if pd.isna(x) else 1)

# Ensure column is integer type
deliveries_df['player_dismissed'] = deliveries_df['player_dismissed'].astype(int)

# Now calculate cumulative wickets per match
wickets_fallen= deliveries_df.groupby('match_id')['player_dismissed'].cumsum()
deliveries_df["wickets_fallen"]=10-wickets_fallen


In [30]:
deliveries_df.head()

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batter,...,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls_left,wickets_fallen
124,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,...,1,NaN,0,1,NaN,NaN,1,221,127,9
125,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,...,1,wides,0,1,NaN,NaN,2,220,128,8
126,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,...,0,NaN,0,1,NaN,NaN,2,220,129,7
127,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,W Jaffer,...,1,NaN,0,1,NaN,NaN,3,219,130,6
128,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,R Dravid,...,1,NaN,0,1,NaN,NaN,4,218,131,5


In [31]:
deliveries_df['crr'] = (deliveries_df['current_score'] * 6) / (120 - deliveries_df['balls_left'])
deliveries_df['rrr'] = (deliveries_df['runs_left'] * 6) / deliveries_df['balls_left']


In [32]:
def result(row):
    return 1 if row['batting_team']== row['winner'] else 0

In [33]:
deliveries_df['result']=deliveries_df.apply(result,axis=1)

In [34]:
# Replace inf, -inf with NaN
deliveries_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows where crr or rrr became NaN
deliveries_df.dropna(subset=['crr', 'rrr'], inplace=True)


In [35]:
final_df=deliveries_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets_fallen','total_runs_x','crr','rrr','result']]

In [36]:
final_df=final_df.sample(final_df.shape[0])

In [37]:
final_df=final_df[final_df['balls_left']!= 0]

In [38]:
final_df

,batting_team,bowling_team,city,runs_left,balls_left,wickets_fallen,total_runs_x,crr,rrr,result
169076,Rajasthan Royals,Kolkata Knight Riders,Kolkata,47,75,-50,149,13.600000,3.760000,1
8119,Delhi Daredevils,Mumbai Indians,Delhi,7,24,-105,176,10.562500,1.750000,1
3998,Delhi Daredevils,Mumbai Indians,Mumbai,33,36,-93,162,9.214286,5.500000,0
101382,Chennai Super Kings,Sunrisers Hyderabad,Mumbai,160,104,-16,178,6.750000,9.230769,1
5503,Deccan Chargers,Kolkata Knight Riders,Hyderabad,141,90,-42,204,12.600000,9.400000,0
...,...,...,...,...,...,...,...,...,...,...
57382,Kolkata Knight Riders,Mumbai Indians,Mumbai,70,35,-91,170,7.058824,12.000000,0
12137,Deccan Chargers,Chennai Super Kings,Durban,80,78,-53,165,12.142857,6.153846,1
89585,Delhi Daredevils,Mumbai Indians,Delhi,177,100,-24,212,10.500000,10.620000,0
106499,Chennai Super Kings,Kolkata Knight Riders,Kolkata,54,52,-73,161,9.441176,6.230769,1


In [39]:
final_df.dropna(inplace= True)
X=final_df.iloc[:,:-1]
y=final_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2,random_state=42)

In [40]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline


# Define categorical and numeric columns
cat_cols = ['batting_team', 'bowling_team', 'city']
num_cols = []  # add your numeric columns here (if any)

# Column transformer — apply proper transformations to each type
trf = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), cat_cols),
    ('num', SimpleImputer(strategy='mean'), num_cols)
], remainder='passthrough')

# Logistic Regression pipeline
lr_pipe = Pipeline(steps=[
    ('preprocessor', trf),
    ('model', LogisticRegression(solver='liblinear'))
])

# Random Forest pipeline
rf_pipe = Pipeline(steps=[
    ('preprocessor', trf),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [41]:
# Fit the models
lr_pipe.fit(X_train, y_train)
rf_pipe.fit(X_train, y_train)

# Make predictions
lr_preds = lr_pipe.predict(X_test)
rf_preds = rf_pipe.predict(X_test)

In [42]:
X_train

,batting_team,bowling_team,city,runs_left,balls_left,wickets_fallen,total_runs_x,crr,rrr
646,Royal Challengers Bangalore,Mumbai Indians,Mumbai,61,60,-70,165,10.400000,6.100000
102869,Sunrisers Hyderabad,Rajasthan Royals,Hyderabad,63,58,-67,198,13.064516,6.517241
129470,Delhi Capitals,Kolkata Knight Riders,Ahmedabad,87,101,-27,154,21.157895,5.168317
44837,Delhi Daredevils,Rajasthan Royals,Jaipur,22,57,-66,141,11.333333,2.315789
28657,Delhi Daredevils,Mumbai Indians,Mumbai,71,40,-85,183,8.400000,10.650000
...,...,...,...,...,...,...,...,...,...
154340,Rajasthan Royals,Chennai Super Kings,Mumbai,46,47,-81,150,8.547945,5.872340
165797,Mumbai Indians,Rajasthan Royals,Mumbai,175,103,-17,212,13.058824,10.194175
56004,Sunrisers Hyderabad,Mumbai Indians,Hyderabad,30,44,-77,129,7.815789,4.090909
17704,Chennai Super Kings,Mumbai Indians,Port Elizabeth,46,45,-80,147,8.080000,6.133333


In [43]:
from sklearn.metrics import accuracy_score
lr_score=accuracy_score(y_test,lr_preds)
rf_score=accuracy_score(y_test,rf_preds)

In [44]:
print("Accuracy score:")
print(f"Logistic regression {lr_score*100}")
print(f"Logistic regression {rf_score*100}")

Accuracy score:
Logistic regression 79.50617283950618
Logistic regression 99.37095825984716


In [45]:
import pickle
pickle.dump(lr_pipe,open('lr_pipe.pkl','wb'))
pickle.dump(rf_pipe,open('rf_pipe.pkl','wb'))

In [46]:
deliveries_df['city'].unique()

array(['Bangalore', 'Delhi', 'Mumbai', 'Kolkata', 'Hyderabad', 'Chennai',
       'Jaipur', 'Cape Town', 'Port Elizabeth', 'Durban', 'Centurion',
       'East London', 'Johannesburg', 'Kimberley', 'Bloemfontein',
       'Ahmedabad', 'Cuttack', 'Nagpur', 'Visakhapatnam', 'Pune',
       'Raipur', 'Ranchi', 'Abu Dhabi', nan, 'Bengaluru', 'Dubai',
       'Sharjah', 'Navi Mumbai', 'Lucknow', 'Guwahati', 'Chandigarh',
       'Dharamsala', 'Mohali'], dtype=object)